In [1]:
import itertools as it
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools as it
from collections import defaultdict
import seaborn as sns
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

In [2]:
path_common_source_trans = '/home/user/Desktop/courseProject/data/train_transaction.csv'
path_common_source_idn = '/home/user/Desktop/courseProject/data/train_identity.csv'
path_to_data_folder = '/home/user/Desktop/courseProject/data/'

In [3]:
clear_data = pd.read_csv(path_to_data_folder + '/clear_and_filled.csv')

In [4]:
clear_data.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_20', 'id_28', 'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=218)

In [5]:
# encoding sources
cat_dum = ['card4', 'card6','ProductCD', 'P_emaildomain', 'R_emaildomain', 'M1', 
           'M2', 'M3', 'M4','M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37'
          , 'id_38', 'DeviceType']
mean_dum = ['id_31', 'DeviceInfo', 'card1', 'card2',  'card3', 'card5', 'addr1', 'addr2']

def get_mean_encoding(df, group, target):
    """Group a Pandas DataFrame by a given column and return the mean encoding of the target variable for that grouping.
    
    Args:
        :param df: Pandas DataFrame.
        :param group: Column to group by. 
        :param target: Target variable column.
        
    Returns:
        Mean for the target variable across the group.
    
    Example: 
        df['mean_encoding_postcode_area'] = get_mean_encoding(df, 'postcode_area', 'attended')
    """
    
    mean_encoded = df.groupby(group)[target].mean()
    return df[group].map(mean_encoded)

In [6]:
#y = clear_data['isFraud']
X = clear_data #.drop('isFraud', axis=1)


# code One Hot Encoding only cat_dum - cols
X = pd.get_dummies(X, drop_first=True, columns=cat_dum)
X_red_mean = X.copy()
X_red_mean.drop(columns=mean_dum)
for col in mean_dum:
    t = get_mean_encoding(X, col, 'isFraud')
    X_red_mean[t.name] = t.values
X = X_red_mean.copy()
ALL = X.copy()
ALL.to_csv(path_to_data_folder + '/clear_and_encoded.csv', index=False)
y = X['isFraud']
X = X.drop('isFraud', axis=1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=123)

### Обучение модели и оценка качества

In [9]:
model = LinearRegression()
model.fit(Xtrain, ytrain)
print(r2_score(ytest, model.predict(Xtest)))
print(r2_score(ytrain, model.predict(Xtrain)))

0.656269754208225
0.6701009381348649


In [10]:
model_l2 = Ridge(alpha=1)
model_l2.fit(Xtrain, ytrain)
print(r2_score(ytest, model_l2.predict(Xtest)))
print(r2_score(ytrain, model_l2.predict(Xtrain)))

0.6562897132118373
0.6700966701383168


In [ ]:
lr_mse = SGDRegressor(loss='squared_loss')
lr_mse.fit(Xtrain, ytrain)
print(r2_score(ytest,lr_mse.predict(Xtest)))
print(r2_score(ytrain,lr_mse.predict(Xtrain)))

/home/user/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(


In [141]:
lr_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0)
lr_mae.fit(Xtrain, ytrain)
print(r2_score(ytest, lr_mae.predict(Xtest)))
print(r2_score(ytrain, r_mae.predict(Xtrain)))

/home/user/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


-1.4216287244226052e+19

### Correlation

In [18]:
for col in X.columns:
    t = y.corr(X.loc[:,col])
    if abs(t) > 0.3:
        print(t, col)

0.34987461047080426 card1
0.4183993462012741 id_17
-0.4090887630471373 id_16_NotFound
-0.4109808695607545 id_35_T
0.37756169158802594 DeviceType_mobile


In [12]:
X.corr()

,Unnamed: 0,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C3,C5,C13,D1,D3,D4,D5,D10,D11,D15,V1,V2,V3,V4,V6,V7,V8,V9,V10,V12,V14,V15,V17,V19,V23,V24,V25,V26,V27,V29,V32,V35,V37,V38,V39,V41,V42,V44,V46,V47,V49,V52,V53,V55,V56,V60,V61,V62,V65,V66,V67,V68,V70,V74,V75,V77,V78,V81,V82,V83,V86,V87,V95,V98,V99,V100,V104,V105,V107,V108,V109,V110,V111,V112,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V129,V130,V131,V135,V136,V169,V170,V171,V172,V173,V174,V175,V176,V180,V181,V184,V185,V186,V187,V188,V189,V191,V194,V195,V199,V200,V204,V205,V207,V208,V209,V210,V214,V215,V217,V220,V221,V223,V224,V226,V227,V228,V229,V230,V234,V238,V240,V241,V242,V246,V247,V248,V250,V252,V255,V258,V260,V261,V262,V264,V267,V268,V270,V274,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V300,V303,V305,V310,V311,V312,V313,V314,V319,V320,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20,id_31,DeviceInfo,card4_discover,card4_mastercard,card4_visa,card6_credit,card6_debit,card6_debit or credit,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmail.com,R_emaildomain_hotmail.de,R_emaildomain_hotmail.es,R_emaildomain_hotmail.fr,R_emaildomain_icloud.com,R_emaildomain_juno.com,R_emaildomain_live.com,R_emaildomain_live.com.mx,R_emaildomain_live.fr,R_emaildomain_mac.com,R_emaildomain_mail.com,R_emaildomain_me.com,R_emaildomain_msn.com,R_emaildomain_netzero.com,R_emaildomain_netzero.net,R_emaildomain_optonline.net,R_emaildomain_outlook.com,R_emaildomain_outlook.es,R_emaildomain_prodigy.net.mx,R_emaildomain_protonmail.com,R_emaildomain_ptd.net,R_emaildomain_q.com,R_emaildomain_roadrunner.com,R_emaildomain_rocketmail.com,R_emaildomain_sbcglobal.net,R_emaildomain_sc.rr.com,R_emaildomain_scranton.edu,R_emaildomain_servicios-ta.com,R_emaildomain_suddenlink.net,R_emaildomain_twc.com,R_emaildomain_verizon.net,R_emaildomain_web.de,R_emaildomain_windstream.net,R_emaildomain_yahoo.co.jp,R_emaildomain_yahoo.co.uk,R_emaildomain_yahoo.com,R_emaildomain_yahoo.com.mx,R_emaildomain_yahoo.de,R_emaildomain_yahoo.es,R_e